In [ ]:
import torch

'''
1. 设置随机数种子
2. 设备
3. 其他参数设置
4. 数据集
5. 模型
6. 定义损失函数和优化器
7. 训练和测试
'''

In [ ]:
# 设置随机数种子
import numpy as np
import random
import os
seed = 42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) # 为当前的GPU设置产生随机数的种子
# torch.cuda.manual_seed_all(seed) # 为所有GPU设置产生随机数的种子
torch.backends.cudnn.deterministic = True # 每次返回的卷积算法将是确定的; 会降低训练速度; 从checkpoints重新开始时会出现意外的结果
# torch.backends.cudnn.benchmark = False # 设置为True, 可以大大提升卷积神经网络的运行速度; 可在网络训练开始前设置

In [ ]:
# 设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 后续的数据以及模型需要.to(device)

In [ ]:
# 其他参数设置: 包括数据集, 模型, 训练等参数

In [ ]:
# 数据集

torch.utils.data.DataLoader()

In [ ]:
# 模型

# 预训练模型
from torch import nn
# 若仅想在模型顶层微调, 则按如下设置:
pretrain_model = None
for param in pretrain_model.parameters():
    param.requires_grad = False
pretrain_model.fc = nn.Linear(pretrain_model.fc.in_features, 100) # 替换顶层来微调

# 下载模型
# 下载整个模型
model = torch.load('model.ckpt', map_location=torch.device('cuda:0'))
# map_location指明此时的device环境; map_location='cpu'
# 如果保存的模型是torch.nn.DataParallel, 则当前的模型也需要是
# model.load_state_dict(checkpoint['state_dict']) 只下载模型参数; model.load_state_dict(torch.load('params.ckpt'))
	# 函数load_state_dict有参数strict, 默认strict=True; 判断参数拷贝过程中是否有unexpected_keys或者missing_keys, 如果有就报错, 代码不能继续执行
	# 如果strict=False, 则会忽略这些细节
	# model.state_dict()返回的是一个OrderDict, 存储了网络结构的名字和对应的参数
# optimizer.load_state_dict(checkpoint['optimizer'])

# 分布式模型
model = torch.nn.DataParallel(model).cuda()
torch.nn.DataParallel(model, device_ids=[0,1,2])

In [ ]:
# 定义损失函数和优化器

# 优化器
optimizer = torch.optim.SGD() 
# 第一个参数可取model.parameters()
# 还有参数lr, weight_decay, momentum
torch.optim.Adam()
# 第一个参数可取model.parameters()
# 还有参数lr, weight_decay
# Adagrad效果比Adam好的多

# 可根据epoch修改学习率
'''
for param_group in optimizer.param_groups:  
	param_group['lr'] = lr
'''

In [ ]:
# 训练和测试

# 训练
# 迭代进行(例如[for images, labels in train_loader]); 若只想显示最终的结果, 则可借助IPython的display.clear_output(wait=True), 置于输出语句之后
'''
1. 前向传播
2. 计算损失loss
3. 将梯度置零: optimizer.zero_grad()
4. 反向传播: loss.backward()
5. 更新参数: optimizer.step()
'''

# 可绘图(针对二维数据)
# X是二维数据, y是标签
from matplotlib import pyplot as plt
def plot_data(X, y, auto=False, zoom=1):
    X = X.cpu()
    y = y.cpu()
    plt.scatter(X.numpy()[:, 0], X.numpy()[:, 1], c=y, s=20, cmap=plt.cm.Spectral)
    plt.axis('square')
    plt.axis(np.array((-1.1, 1.1, -1.1, 1.1)) * zoom)
    if auto is True: plt.axis('equal')
    plt.axis('off')

    _m, _c = 0, '.15'
    plt.axvline(0, ymin=_m, color=_c, lw=1, zorder=0)
    plt.axhline(0, xmin=_m, color=_c, lw=1, zorder=0)
def plot_model(X, y, model):
    model.cpu()
    mesh = np.arange(-1.1, 1.1, 0.01)
    xx, yy = np.meshgrid(mesh, mesh)
    with torch.no_grad():
        data = torch.from_numpy(np.vstack((xx.reshape(-1), yy.reshape(-1))).T).float()
        Z = model(data).detach()
    Z = np.argmax(Z, axis=1).reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.3)
    plot_data(X, y)

# 存储模型
# 存储整个模型
torch.save(model, 'model.ckpt') 
# 只保存模型参数
torch.save(model.state_dict(), 'params.ckpt')
# 可保存自定义字典, 文件名如ckpt.pth.tar
	# 一般包括模型, 优化器, 迭代轮次epoch(便于中断后恢复训练后), 最佳acc
import shutil
start_epoch = 0
resume = 0
num_epochs = None
current_acc = None
if resume: # resume为参数, 第一次训练时设为0, 中断再训练时设为1
    model_path = os.path.join('model', 'best_checkpoint.pth.tar')
    assert os.path.isfile(model_path)
    checkpoint = torch.load(model_path)
    best_acc = checkpoint['best_acc']
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print('Load checkpoint at epoch {}.'.format(start_epoch))
    print('Best accuracy so far {}.'.format(best_acc))

for epoch in range(start_epoch, num_epochs): 
    ... 

    # Test the model
    ...

    # save checkpoint
    is_best = current_acc > best_acc
    best_acc = max(current_acc, best_acc)
    checkpoint = {
        'best_acc': best_acc,
        'epoch': epoch + 1,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    model_path = os.path.join('model', 'checkpoint.pth.tar')
    best_model_path = os.path.join('model', 'best_checkpoint.pth.tar')
    torch.save(checkpoint, model_path)
    if is_best:
        shutil.copy(model_path, best_model_path)